# Contact partitioning with public schools in Austin MSA

In [1]:
import logging
gpd_logger = logging.getLogger('geopandas')
gpd_logger.setLevel(logging.WARNING)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
fi_logger = logging.getLogger('fiona')
fi_logger.setLevel(logging.WARNING)
import pandas as pd
import numpy as np
import xarray as xr
import xsimlab as xs
from episimlab.partition import partition
#from episimlab.setup.coords import InitDefaultCoords
import multiprocessing as mp
from datetime import datetime
import geopandas as gpd
import descartes
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import datetime
from collections import defaultdict
import contextily as ctx
import json

## SABS shapefiles (2015-2016)

In [2]:
primary = gpd.read_file('/Users/kpierce/COVID19/SchoolCatchmentDemo/SABS_1516_SchoolLevels/SABS_1516_Primary.shp')

In [3]:
primary_tx = primary[primary['stAbbrev'] == 'TX']

In [4]:
primary_tx.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,geometry
36034,None,480000101145,CROSBYTON EL,4800001,PK,05,1,TX,0,231164.650104,1.768275e+09,1,0,"POLYGON ((-11281346.371 3987964.176, -11281343..."
36035,None,480000600274,ASPERMONT EL,4800006,PK,05,1,TX,0,246901.962200,3.005729e+09,1,0,"POLYGON ((-11189642.460 3898709.547, -11189638..."
36036,Dripping Springs,480000801507,DRIPPING SPRINGS EL,4800008,PK,05,0,TX,0,109636.791042,2.125308e+08,1,0,"MULTIPOLYGON (((-10922523.357 3551547.378, -10..."
36037,Walnut Springs,480000806989,WALNUT SPRINGS EL,4800008,PK,05,0,TX,0,159876.915178,4.116677e+08,1,0,"POLYGON ((-10926141.469 3546621.811, -10925182..."
36038,Rooster Springs,480000811280,ROOSTER SPRINGS EL,4800008,PK,05,0,TX,0,42288.398204,5.389171e+07,1,0,"POLYGON ((-10901471.558 3526669.331, -10901437..."


In [5]:
middle = gpd.read_file('/Users/kpierce/COVID19/SchoolCatchmentDemo/SABS_1516_SchoolLevels/SABS_1516_Middle.shp')
middle_tx = middle[middle['stAbbrev'] == 'TX']

In [6]:
high = gpd.read_file('/Users/kpierce/COVID19/SchoolCatchmentDemo/SABS_1516_SchoolLevels/SABS_1516_High.shp')
high_tx = high[high['stAbbrev'] == 'TX']

In [7]:
primary.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
middle.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
high.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
tx_school_shapes_all = pd.concat([high_tx, middle_tx, primary_tx])
tx_school_shapes = tx_school_shapes_all[['ncessch', 'geometry']]

In [11]:
tx_school_shapes_all.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,geometry
11799,DRIPPING SPRINGS H S,480000801508,DRIPPING SPRINGS H S,4800008,09,12,0,TX,0,162774.400948,6.780902e+08,3,0,"POLYGON ((-10922523.357 3551547.378, -10922512..."
11800,None,480000906198,WIMBERLEY H S,4800009,09,12,1,TX,0,123681.690416,4.646886e+08,3,0,"POLYGON ((-10937603.921 3506173.262, -10936378..."
11801,HHS 12-13,480001005244,JACK C HAYS H S,4800010,09,12,0,TX,0,159649.428183,5.160653e+08,3,0,"POLYGON ((-10895818.506 3522395.778, -10895765..."
11802,LHS 12-13,480001009624,LEHMAN H S,4800010,09,12,0,TX,0,110784.816354,2.362902e+08,3,0,"POLYGON ((-10891508.953 3507662.991, -10891431..."
11803,None,480001201431,CHISUM H S,4800012,09,12,1,TX,0,292003.989257,7.633954e+08,3,0,"POLYGON ((-10670741.223 3978573.715, -10670718..."


In [12]:
tx_school_shapes_all['gshi'].unique()

array(['12', '08', '06', '07', '09', '05', '01', '04', '02', 'KG', '03'],
      dtype=object)

In [13]:
tx_school_shapes.head()

,ncessch,geometry
11799,480000801508,"POLYGON ((-10922523.357 3551547.378, -10922512..."
11800,480000906198,"POLYGON ((-10937603.921 3506173.262, -10936378..."
11801,480001005244,"POLYGON ((-10895818.506 3522395.778, -10895765..."
11802,480001009624,"POLYGON ((-10891508.953 3507662.991, -10891431..."
11803,480001201431,"POLYGON ((-10670741.223 3978573.715, -10670718..."


## TEA attendance data

Download statewide reports of campus enrollment from TEA: https://rptsvr1.tea.texas.gov/adhocrpt/adste.html

Download school point locations from TEA: 

- https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/school2020to2021/explore
- https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/current-schools2019to2020/explore

In [14]:
import re

In [15]:
schools2019 = gpd.read_file('/Users/kpierce/COVID19/SchoolCatchmentDemo/School2019to2020/Current_Schools.shp')
schools2020 = gpd.read_file('/Users/kpierce/COVID19/SchoolCatchmentDemo/School2020to2021/School2020to2021.shp')

In [45]:
schools2019.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
schools2019_short = schools2019[[
    'District_N', 'District_1', 'County_Nam', 'Site_Zip5',
    'School_Nam', 'School_Num', 'Grade_Leve', 'School_Enr', 'X', 'Y'
]]
schools2019_short.columns = ['DistrictNumber', 'DistrictName', 'County', 'Zip', 'SchoolName', 'SchoolNumber', 'GradeLevel', 'SchoolEnrollment', 'x', 'y']

In [17]:
schools2019_short['DistrictNumber'] = [re.sub("'", "", i) for i in schools2019_short['DistrictNumber']]
schools2019_short['SchoolNumber'] = [re.sub("'", "", i) for i in schools2019_short['SchoolNumber']]

/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_39221/2663935558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools2019_short['DistrictNumber'] = [re.sub("'", "", i) for i in schools2019_short['DistrictNumber']]
/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_39221/2663935558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools2019_short['SchoolNumber'] = [re.sub("'", "", i) for i in schools2019_short['SchoolNumber']]


In [18]:
schools2019_short.head()

,DistrictNumber,DistrictName,County,Zip,SchoolName,SchoolNumber,GradeLevel,SchoolEnrollment,x,y
0,001902,CAYUGA ISD,ANDERSON COUNTY,75861.0,CAYUGA H S,001902001,High School,179.0,-95.924741,31.922265
1,001902,CAYUGA ISD,ANDERSON COUNTY,75861.0,CAYUGA MIDDLE,001902041,Middle,119.0,-95.924741,31.922265
2,001902,CAYUGA ISD,ANDERSON COUNTY,75861.0,CAYUGA EL,001902103,Elementary,266.0,-95.924741,31.922265
3,001903,ELKHART ISD,ANDERSON COUNTY,75801.0,ELKHART DAEP,001903002,Elementary/Secondary,2.0,-95.638329,31.769995
4,001906,NECHES ISD,ANDERSON COUNTY,75803.0,NECHES H S,001906002,High School,107.0,-95.476226,31.871807


Extract the Austin-Round Rock MSA by county name

In [19]:
atx_schools_2019 = schools2019_short[schools2019_short['County'].isin(set(['TRAVIS COUNTY', 'WILLIAMSON COUNTY', 'HAYS COUNTY', 'BASTROP COUNTY', 'CALDWELL COUNTY']))]
atx_schools_2019 = gpd.GeoDataFrame(
    atx_schools_2019,
    geometry=gpd.points_from_xy(atx_schools_2019['x'], atx_schools_2019['y']),
    crs=schools2019.crs
)

Check that all the geometries are valid and remove those that are not

In [20]:
atx_schools_2019_complete = atx_schools_2019[~(atx_schools_2019.geometry.is_empty | atx_schools_2019.geometry.isna())]

In [21]:
print(atx_schools_2019.shape)
print(atx_schools_2019_complete.shape)

(625, 11)
(625, 11)


In [22]:
atx_schools_2019_valid = atx_schools_2019[atx_schools_2019.is_valid]

In [23]:
print(atx_schools_2019.shape)
print(atx_schools_2019_valid.shape)

(625, 11)
(624, 11)


In [24]:
tx_school_shapes_complete = tx_school_shapes[~(tx_school_shapes.geometry.is_empty | tx_school_shapes.geometry.isna())]

In [25]:
print(tx_school_shapes.shape)
print(tx_school_shapes_complete.shape)

(6270, 2)
(6270, 2)


In [26]:
tx_school_shapes = tx_school_shapes.to_crs(atx_schools_2019.crs)

In [27]:
tx_school_shapes_valid = tx_school_shapes[tx_school_shapes.is_valid]

In [28]:
print(tx_school_shapes.shape)
print(tx_school_shapes_valid.shape)

(6270, 2)
(6235, 2)


## Merge 2019 data

Perform the join with the clean ATX schools data, and ignore the self-intersections in the TX shapefiles. ATX schools must be broken down by grade level, as grade level catchments overlap.

In [29]:
atx_schools_2019_valid['GradeLevel'].unique()

array(['High School', 'Elementary', 'Junior High', 'Middle',
       'Elementary/Secondary'], dtype=object)

In [46]:
high_tx.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [30]:
data_map = {
    'High School': high_tx,
    'Elementary': primary_tx,
    'Junior High': middle_tx,
    'Middle': middle_tx,
    'Elementary/Secondary': pd.concat([middle_tx, primary_tx])
}

In [31]:
join_data = []
for level in atx_schools_2019_valid['GradeLevel'].unique():
    combine_data = data_map[level]
    combine_data = combine_data[['ncessch', 'geometry']].to_crs(atx_schools_2019.crs)
    level_data = atx_schools_2019_valid[atx_schools_2019_valid['GradeLevel'] == level]
    join_data.append(gpd.sjoin(level_data, combine_data, how='left'))
all_2019_joins = pd.concat(join_data)

In [32]:
all_2019_joins.head()

,DistrictNumber,DistrictName,County,Zip,SchoolName,SchoolNumber,GradeLevel,SchoolEnrollment,x,y,geometry,index_right,ncessch
27,011901,BASTROP ISD,BASTROP COUNTY,78612.0,CEDAR CREEK H S,011901002,High School,1583.0,-97.471643,30.138354,POINT (-97.47164 30.13835),11863.0,480957012272
28,011901,BASTROP ISD,BASTROP COUNTY,78602.0,COLORADO RIVER COLLEGIATE ACADEMY,011901005,High School,206.0,-97.315166,30.118527,POINT (-97.31517 30.11853),11862.0,480957000409
29,011901,BASTROP ISD,BASTROP COUNTY,78602.0,GENESIS H S,011901022,High School,115.0,-97.315166,30.118527,POINT (-97.31517 30.11853),11862.0,480957000409
37,011902,ELGIN ISD,BASTROP COUNTY,78621.0,PHOENIX H S,011902004,High School,50.0,-97.377919,30.347799,POINT (-97.37792 30.34780),12134.0,481836001728
41,011905,MCDADE ISD,BASTROP COUNTY,78650.0,MCDADE JJAEP,011905011,High School,0.0,-97.239564,30.283470,POINT (-97.23956 30.28347),NaN,NaN


In [33]:
final_2019_schools = pd.merge(all_2019_joins.drop('geometry', axis=1), tx_school_shapes, on='ncessch', how='left')

In [34]:
final_2019_schools.head()

,DistrictNumber,DistrictName,County,Zip,SchoolName,SchoolNumber,GradeLevel,SchoolEnrollment,x,y,index_right,ncessch,geometry
0,011901,BASTROP ISD,BASTROP COUNTY,78612.0,CEDAR CREEK H S,011901002,High School,1583.0,-97.471643,30.138354,11863.0,480957012272,"POLYGON ((-97.39664 30.21163, -97.39672 30.211..."
1,011901,BASTROP ISD,BASTROP COUNTY,78602.0,COLORADO RIVER COLLEGIATE ACADEMY,011901005,High School,206.0,-97.315166,30.118527,11862.0,480957000409,"POLYGON ((-97.12109 30.27806, -97.12075 30.277..."
2,011901,BASTROP ISD,BASTROP COUNTY,78602.0,GENESIS H S,011901022,High School,115.0,-97.315166,30.118527,11862.0,480957000409,"POLYGON ((-97.12109 30.27806, -97.12075 30.277..."
3,011902,ELGIN ISD,BASTROP COUNTY,78621.0,PHOENIX H S,011902004,High School,50.0,-97.377919,30.347799,12134.0,481836001728,"POLYGON ((-97.43931 30.44814, -97.43540 30.446..."
4,011905,MCDADE ISD,BASTROP COUNTY,78650.0,MCDADE JJAEP,011905011,High School,0.0,-97.239564,30.283470,NaN,NaN,None


Schools added after the 2015-2016 SABS district survey:

In [47]:
final_2019_schools['has_catchment'] = [True if i != None else False for i in final_2019_schools['geometry']]

In [48]:
final_2019_schools['has_catchment'].value_counts()

True     727
False    191
Name: has_catchment, dtype: int64

In [49]:
final_2019_schools_locations = final_2019_schools.drop('geometry', axis=1)
final_2019_schools_locations = gpd.GeoDataFrame(
    final_2019_schools_locations,
    geometry=gpd.points_from_xy(final_2019_schools_locations['x'], final_2019_schools_locations['y']),
    crs=final_2019_schools.crs
)

In [50]:
final_2019_schools_locations.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_joined_with_2015_2016_catchments_points_only.shp')


/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_39221/1851134514.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_2019_schools_locations.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_joined_with_2015_2016_catchments_points_only.shp')


In [35]:
new_2019 = final_2019_schools[final_2019_schools['geometry'].isna()]
with_shape = final_2019_schools[~final_2019_schools['geometry'].isna()]

print(new_2019.shape)
print(with_shape.shape)

new_2019['County'].value_counts()

(191, 13)
(727, 13)


TRAVIS COUNTY        183
WILLIAMSON COUNTY      3
BASTROP COUNTY         2
CALDWELL COUNTY        2
HAYS COUNTY            1
Name: County, dtype: int64

In [43]:
with_shape['County'].value_counts()

TRAVIS COUNTY        356
WILLIAMSON COUNTY    243
HAYS COUNTY           54
CALDWELL COUNTY       38
BASTROP COUNTY        36
Name: County, dtype: int64

In [40]:
new_2019_locations = new_2019.drop('geometry', axis=1)
new_2019_locations = gpd.GeoDataFrame(
    new_2019_locations,
    geometry=gpd.points_from_xy(new_2019_locations['x'], new_2019_locations['y']),
    crs=final_2019_schools.crs
)

In [41]:
new_2019_locations.head()

,DistrictNumber,DistrictName,County,Zip,SchoolName,SchoolNumber,GradeLevel,SchoolEnrollment,x,y,index_right,ncessch,geometry
4,011905,MCDADE ISD,BASTROP COUNTY,78650.0,MCDADE JJAEP,011905011,High School,0.0,-97.239564,30.283470,NaN,NaN,POINT (-97.23956 30.28347)
9,227506,UNIVERSITY OF TEXAS AT AUSTIN H S,TRAVIS COUNTY,78722.0,UNIVERSITY OF TEXAS AT AUSTIN H S,227506001,High School,0.0,-97.723570,30.287331,NaN,NaN,POINT (-97.72357 30.28733)
10,227806,UNIVERSITY OF TEXAS UNIVERSITY CHARTER SCHOOL,TRAVIS COUNTY,78705.0,UNIVERSITY H S,227806043,High School,17.0,-97.739276,30.283317,NaN,NaN,POINT (-97.73928 30.28332)
11,227817,CEDARS INTERNATIONAL ACADEMY,TRAVIS COUNTY,78752.0,CEDARS ACADEMY NEXT GENERATION H S AT HIGHLAND,227817001,High School,148.0,-97.706961,30.328561,NaN,NaN,POINT (-97.70696 30.32856)
12,227820,KIPP TEXAS PUBLIC SCHOOLS,TRAVIS COUNTY,78754.0,KIPP AUSTIN COLLEGIATE,227820001,High School,718.0,-97.684886,30.334998,NaN,NaN,POINT (-97.68489 30.33500)


In [44]:
final_2019_schools.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_joined_with_2015_2016_catchments.shp')

/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_39221/3453944136.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_2019_schools.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_joined_with_2015_2016_catchments.shp')


In [42]:
new_2019_locations.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_without_Catchments.shp')

/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_39221/3697970143.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_2019_locations.to_file('/Users/kpierce/COVID19/austin-spatial-prev/2019_Schools_without_Catchments.shp')


## Merge 2020 data

In [36]:
schools2020_short = schools2020[[
    'District_N', 'District_1', 'County_Nam', 'Site_Zip5', 'Organiza_2', 'Campus_Cla', 'Enrollment', 'geometry'
]]
schools2020_short.columns = ['DistrictName', 'DistrictNumber', 'County', 'Zip', 'SchoolNumber', 'GradeLevel', 'SchoolEnrollment', 'geometry']

In [37]:
schools2020_short.head()

,DistrictName,DistrictNumber,County,Zip,SchoolNumber,GradeLevel,SchoolEnrollment,geometry
0,WESLACO ISD,108913,HIDALGO COUNTY,78596,108913118,Elementary,522,POINT (-97.99481 26.15694)
1,DONNA ISD,108902,HIDALGO COUNTY,78537,108902006,High School,146,POINT (-98.04175 26.17158)
2,WYLIE ISD,043914,COLLIN COUNTY,75098,043914005,None,0,POINT (-96.54049 33.00424)
3,ROUND ROCK ISD,246909,WILLIAMSON COUNTY,78681,246909014,None,0,POINT (-97.69693 30.51184)
4,COLLEGE STATION ISD,021901,BRAZOS COUNTY,77840,021901001,High School,1840,POINT (-96.31535 30.59055)


In [38]:
atx_schools_2020 = schools2020_short[schools2020_short['County'].isin(set(['TRAVIS COUNTY', 'WILLIAMSON COUNTY', 'HAYS COUNTY', 'BASTROP COUNTY', 'CALDWELL COUNTY']))]

In [39]:
atx_schools_2020_complete = atx_schools_2020[~(atx_schools_2020.geometry.is_empty | atx_schools_2020.geometry.isna())]

In [40]:
print(atx_schools_2020.shape)
print(atx_schools_2020_complete.shape)

(644, 8)
(644, 8)


In [41]:
assert atx_schools_2020_complete[atx_schools_2020_complete.is_valid].shape == atx_schools_2020_complete.shape

In [42]:
atx_schools_2020_complete.head()

,DistrictName,DistrictNumber,County,Zip,SchoolNumber,GradeLevel,SchoolEnrollment,geometry
3,ROUND ROCK ISD,246909,WILLIAMSON COUNTY,78681,246909014,None,0,POINT (-97.69693 30.51184)
60,LEANDER ISD,246913,WILLIAMSON COUNTY,78613,246913102,Elementary,448,POINT (-97.82981 30.48807)
77,EANES ISD,227909,TRAVIS COUNTY,78746,227909005,High School,14,POINT (-97.80145 30.27286)
100,AUSTIN ISD,227901,TRAVIS COUNTY,78748,227901017,High School,2752,POINT (-97.80212 30.15064)
151,AUSTIN ISD,227901,TRAVIS COUNTY,78741,227901101,Elementary,502,POINT (-97.69215 30.23608)


In [43]:
atx_schools_2020_complete['GradeLevel'].unique()

array([None, 'Elementary', 'High School', 'Elem/Secondary',
       'Middle School'], dtype=object)

In [44]:
data_map_2020 = {
    'High School': high_tx,
    'Elementary': primary_tx,
    'Middle School': middle_tx,
    'Elem/Secondary': pd.concat([middle_tx, primary_tx]),
}

In [45]:
join_data_2020 = []
for level in data_map_2020.keys():
    combine_data = data_map_2020[level]
    combine_data = combine_data[['ncessch', 'geometry']].to_crs(atx_schools_2020_complete.crs)
    level_data = atx_schools_2020_complete[atx_schools_2020_complete['GradeLevel'] == level]
    join_data_2020.append(gpd.sjoin(level_data, combine_data, how='left'))
all_2020_joins = pd.concat(join_data_2020)

In [46]:
all_2020_joins.head()

,DistrictName,DistrictNumber,County,Zip,SchoolNumber,GradeLevel,SchoolEnrollment,geometry,index_right,ncessch
77,EANES ISD,227909,TRAVIS COUNTY,78746,227909005,High School,14,POINT (-97.80145 30.27286),12099.0,481776001549
100,AUSTIN ISD,227901,TRAVIS COUNTY,78748,227901017,High School,2752,POINT (-97.80212 30.15064),NaN,NaN
239,AUSTIN ISD,227901,TRAVIS COUNTY,78759,227901009,High School,2216,POINT (-97.75410 30.37645),NaN,NaN
276,UNIVERSITY OF TEXAS UNIVERSITY CHARTER SCHOOL,227806,TRAVIS COUNTY,78628,227806017,High School,8,POINT (-97.67799 30.71905),12318.0,482460002756
397,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825,TRAVIS COUNTY,78723,227825004,High School,520,POINT (-97.67684 30.32476),12454.0,482889011568


In [47]:
final_2020_schools = pd.merge(all_2020_joins.drop('geometry', axis=1), tx_school_shapes, on='ncessch', how='left')

In [48]:
new_2020 = final_2020_schools[final_2020_schools['geometry'].isna()]
with_shape_2020 = final_2020_schools[~final_2020_schools['geometry'].isna()]

print(new_2020.shape)
print(with_shape_2020.shape)

new_2020['County'].value_counts()

(179, 10)
(608, 10)


TRAVIS COUNTY        172
BASTROP COUNTY         2
WILLIAMSON COUNTY      2
CALDWELL COUNTY        2
HAYS COUNTY            1
Name: County, dtype: int64

# Combine 2019 and 2020 school catchment files

In [49]:
final_2019_schools.head()

,DistrictNumber,DistrictName,County,Zip,SchoolName,SchoolNumber,GradeLevel,SchoolEnrollment,x,y,index_right,ncessch,geometry
0,011901,BASTROP ISD,BASTROP COUNTY,78612.0,CEDAR CREEK H S,011901002,High School,1583.0,-97.471643,30.138354,11863.0,480957012272,"POLYGON ((-97.39664 30.21163, -97.39672 30.211..."
1,011901,BASTROP ISD,BASTROP COUNTY,78602.0,COLORADO RIVER COLLEGIATE ACADEMY,011901005,High School,206.0,-97.315166,30.118527,11862.0,480957000409,"POLYGON ((-97.12109 30.27806, -97.12075 30.277..."
2,011901,BASTROP ISD,BASTROP COUNTY,78602.0,GENESIS H S,011901022,High School,115.0,-97.315166,30.118527,11862.0,480957000409,"POLYGON ((-97.12109 30.27806, -97.12075 30.277..."
3,011902,ELGIN ISD,BASTROP COUNTY,78621.0,PHOENIX H S,011902004,High School,50.0,-97.377919,30.347799,12134.0,481836001728,"POLYGON ((-97.43931 30.44814, -97.43540 30.446..."
4,011905,MCDADE ISD,BASTROP COUNTY,78650.0,MCDADE JJAEP,011905011,High School,0.0,-97.239564,30.283470,NaN,NaN,None


In [50]:
final_2020_schools.head()

,DistrictName,DistrictNumber,County,Zip,SchoolNumber,GradeLevel,SchoolEnrollment,index_right,ncessch,geometry
0,EANES ISD,227909,TRAVIS COUNTY,78746,227909005,High School,14,12099.0,481776001549,"POLYGON ((-97.84572 30.34144, -97.84520 30.340..."
1,AUSTIN ISD,227901,TRAVIS COUNTY,78748,227901017,High School,2752,NaN,NaN,None
2,AUSTIN ISD,227901,TRAVIS COUNTY,78759,227901009,High School,2216,NaN,NaN,None
3,UNIVERSITY OF TEXAS UNIVERSITY CHARTER SCHOOL,227806,TRAVIS COUNTY,78628,227806017,High School,8,12318.0,482460002756,"POLYGON ((-97.75078 30.79988, -97.74573 30.801..."
4,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825,TRAVIS COUNTY,78723,227825004,High School,520,12454.0,482889011568,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."


In [51]:
school_names = final_2019_schools[['SchoolName', 'SchoolNumber']]
final_2020_schools_named = pd.merge(final_2020_schools, school_names, on='SchoolNumber', how='left').drop_duplicates()

In [52]:
final_2020_schools_named.head()

,DistrictName,DistrictNumber,County,Zip,SchoolNumber,GradeLevel,SchoolEnrollment,index_right,ncessch,geometry,SchoolName
0,EANES ISD,227909,TRAVIS COUNTY,78746,227909005,High School,14,12099.0,481776001549,"POLYGON ((-97.84572 30.34144, -97.84520 30.340...",ADULT TRANSITION SERVICES
1,AUSTIN ISD,227901,TRAVIS COUNTY,78748,227901017,High School,2752,NaN,NaN,None,AKINS H S
2,AUSTIN ISD,227901,TRAVIS COUNTY,78759,227901009,High School,2216,NaN,NaN,None,ANDERSON H S
3,UNIVERSITY OF TEXAS UNIVERSITY CHARTER SCHOOL,227806,TRAVIS COUNTY,78628,227806017,High School,8,12318.0,482460002756,"POLYGON ((-97.75078 30.79988, -97.74573 30.801...",ANNUNCIATION MATERNITY HOME
6,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825,TRAVIS COUNTY,78723,227825004,High School,520,12454.0,482889011568,"POLYGON ((-97.68606 30.33311, -97.68333 30.337...",AUSTIN ACHIEVE H S


In [53]:
final_2019_schools['YEAR'] = '2019-2020'
final_2020_schools_named['YEAR'] = '2020-2021'
final_school_catchments = pd.concat([final_2019_schools, final_2020_schools_named])

In [54]:
final_school_catchments = final_school_catchments.to_crs(atx_schools_2019.crs)

In [55]:
final_school_catchments.to_file('/Users/kpierce/COVID19/austin-spatial-prev/AustinRoundRockMSA_SchoolCatchments_2019_2021_20211203.shp')


/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_65476/3169516442.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_school_catchments.to_file('/Users/kpierce/COVID19/austin-spatial-prev/AustinRoundRockMSA_SchoolCatchments_2019_2021_20211203.shp')


# Merge in attendance data

In [56]:
attendance2019_2020 = pd.read_csv(
    '/Users/kpierce/COVID19/SchoolCatchmentDemo/Enrollment Report_Statewide_Campuses_Grade_2019-2020.csv',
    dtype={'CAMPUS': str, 'DISTRICT': str, 'REGION': str}
)
atx_msa_2019_2020 = attendance2019_2020[attendance2019_2020['COUNTY NAME'].isin(['TRAVIS COUNTY', 'WILLIAMSON COUNTY', 'HAYS COUNTY', 'BASTROP COUNTY', 'CALDWELL COUNT'])]                     


In [57]:
attendance2020_2021 = pd.read_csv(
    '/Users/kpierce/COVID19/SchoolCatchmentDemo/Enrollment Report_Statewide_Campuses_Grade_2020-2021.csv',
    dtype={'CAMPUS': str, 'DISTRICT': str, 'REGION': str}
)
atx_msa_2020_2021 = attendance2020_2021[attendance2020_2021['COUNTY NAME'].isin(['TRAVIS COUNTY', 'WILLIAMSON COUNTY', 'HAYS COUNTY', 'BASTROP COUNTY', 'CALDWELL COUNT'])]                     


In [58]:
atx_msa_enrollment = pd.concat([atx_msa_2019_2020, atx_msa_2020_2021])

In [59]:
atx_msa_enrollment.head()

,YEAR,REGION,COUNTY NAME,DISTRICT,DISTRICT NAME,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT
2747,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119
2748,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81
2749,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114
2750,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112
2751,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130


In [60]:
atx_msa_enroll = pd.merge(
    atx_msa_enrollment, final_school_catchments[['YEAR', 'SchoolNumber', 'geometry']].drop_duplicates(), 
    left_on=['YEAR', 'CAMPUS'], right_on=['YEAR', 'SchoolNumber'], 
    how='left'
)


In [61]:
atx_msa_enroll_shp = gpd.GeoDataFrame(atx_msa_enroll)

In [62]:
atx_msa_enroll_shp.to_file('/Users/kpierce/COVID19/austin-spatial-prev/AustinRoundRockMSA_SchoolCatchments_GradeEnrollment_2019_2021_20211203.shp')


/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_65476/2629999050.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  atx_msa_enroll_shp.to_file('/Users/kpierce/COVID19/austin-spatial-prev/AustinRoundRockMSA_SchoolCatchments_GradeEnrollment_2019_2021_20211203.shp')


In [63]:
atx_msa_enroll_shp.head()

,YEAR,REGION,COUNTY NAME,DISTRICT,DISTRICT NAME,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT,SchoolNumber,geometry
0,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119,227825003,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."
1,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81,227825003,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."
2,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114,227825003,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."
3,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112,227825003,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."
4,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130,227825003,"POLYGON ((-97.68606 30.33311, -97.68333 30.337..."


# School travel based on areal weighted interpolation

Map the grade levels (column `GRADE`) to ages, and combine with mobility data in a contact partitioning process.

In [64]:
tx_zcta = gpd.read_file('/Users/kpierce/CooksProTX/spatial/tigris/texas_zcta/texas_zcta_2019.shp')

In [65]:
tx_zcta = tx_zcta.to_crs('EPSG:3857')

In [66]:
tx_zcta.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [67]:
tx_zcta['zcta_area'] = tx_zcta.area

In [74]:
atx_msa_enroll = gpd.GeoDataFrame(atx_msa_enroll).to_crs(high.crs) # all the shapefiles for boundaries have the same CRS
atx_msa_enroll.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [75]:
atx_msa_enroll['catchment_area'] = atx_msa_enroll.area

In [76]:
atx_msa_enroll_valid = atx_msa_enroll[atx_msa_enroll.is_valid]

In [183]:
atx_school_zcta = gpd.overlay(atx_msa_enroll_valid, tx_zcta, how='intersection')

In [184]:
atx_school_zcta.columns

Index(['YEAR', 'REGION_1', 'COUNTY NAME', 'DISTRICT', 'DISTRICT NAME',
       'CAMPUS', 'CAMPUS NAME', 'GRADE GROUP NAME', 'GRADE', 'ENROLLMENT',
       'SchoolNumber', 'catchment_area', 'ZCTA5CE10', 'GEOID10', 'CLASSFP10',
       'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10',
       'INTPTLON10', 'index_righ', 'REGION_2', 'DIVISION', 'STATEFP',
       'STATENS', 'GEOID', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT',
       'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'zcta_area', 'geometry'],
      dtype='object')

In [188]:
atx_school_zcta = atx_school_zcta[['YEAR', 'COUNTY NAME', 'DISTRICT', 'DISTRICT NAME',
       'CAMPUS', 'CAMPUS NAME', 'GRADE GROUP NAME', 'GRADE', 'ENROLLMENT',
       'catchment_area', 'ZCTA5CE10', 'zcta_area', 'geometry']]

In [189]:
atx_school_zcta.head()

,YEAR,COUNTY NAME,DISTRICT,DISTRICT NAME,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT,catchment_area,ZCTA5CE10,zcta_area,geometry
0,2019-2020,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579..."
1,2019-2020,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579..."
2,2019-2020,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579..."
3,2019-2020,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579..."
4,2019-2020,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579..."


Some ZCTAs overlap school catchments but do not have children.

In [162]:
census = pd.read_csv('/Users/kpierce/COVID19/SchoolCatchmentDemo/2019_TX_ZCTA_age_populations.csv', dtype={'GEOID': str})

In [163]:
census.dtypes

Unnamed: 0     int64
GEOID         object
age           object
estimate       int64
dtype: object

In [164]:
census.head()

,Unnamed: 0,GEOID,age,estimate
0,1,75001,10 to 14 years,401
1,2,75001,15 to 17 years,277
2,3,75001,18 and 19 years,147
3,4,75001,20 years,180
4,5,75001,21 years,163


In [165]:
age_groups = {
    "10 to 14 years": "10-14",
    "15 to 17 years": "15-17",
    "18 and 19 years": "18-49",
    "20 years": "18-49",
    "21 years": "18-49",
    "22 to 24 years": "18-49",
    "25 to 29 years": "18-49",
    "30 to 34 years": "18-49",
    "35 to 39 years": "18-49",
    "40 to 44 years": "18-49",
    "45 to 49 years": "18-49",
    "5 to 9 years": "5-9",
    "50 to 54 years": "50-64",
    "55 to 59 years": "50-64",
    "60 and 61 years": "50-64",
    "62 to 64 years": "50-64",
    "65 and 66 years": "50-64",
    "67 to 69 years": "50-64",
    "70 to 74 years": "50-64",
    "75 to 79 years": "50-64",
    "80 to 84 years": "50-64",
    "85 years and over": "65+",
    "Under 5 years": "0-4"
}

In [166]:
census['age_group'] = [age_groups[i] for i in census['age']]

In [167]:
census.head()

,Unnamed: 0,GEOID,age,estimate,age_group
0,1,75001,10 to 14 years,401,10-14
1,2,75001,15 to 17 years,277,15-17
2,3,75001,18 and 19 years,147,18-49
3,4,75001,20 years,180,18-49
4,5,75001,21 years,163,18-49


In [168]:
census = census.groupby(['GEOID', 'age_group']).sum('estimate').drop('Unnamed: 0', axis=1).reset_index()

In [169]:
census.head()

,GEOID,age_group,estimate
0,75001,0-4,794
1,75001,10-14,401
2,75001,15-17,277
3,75001,18-49,9420
4,75001,5-9,726


In [172]:
len(census['GEOID'].unique())

1935

In [175]:
census_no_0_4 = census[(census['age_group'] == '0-4') & (census['estimate'] < 1)]
len(census_no_0_4['GEOID'].unique())

191

In [ ]:
census['has_elem'] = [False if i in set(census_no_0_4) else True for i in census[]]

In [177]:
census_no_5_9 = census[(census['age_group'] == '5-9') & (census['estimate'] < 1)]
len(census_no_5_9['GEOID'].unique())

178

In [178]:
census_no_10_14 = census[(census['age_group'] == '10-14') & (census['estimate'] < 1)]
len(census_no_10_14['GEOID'].unique())

175

In [180]:
census_no_15_17 = census[(census['age_group'] == '15-17') & (census['estimate'] < 1)]
len(census_no_15_17['GEOID'].unique())

192

For each grade level, for each zip code, we need to drop out the ZCTAs that don't contribute children from the percentage calculation.

In [78]:
atx_school_zcta['overlap_area'] = atx_school_zcta.area
atx_school_zcta['pct_zcta_to_school'] = atx_school_zcta['overlap_area'] / atx_school_zcta['zcta_area']
atx_school_zcta['pct_school_from_zcta'] = atx_school_zcta['overlap_area'] / atx_school_zcta['catchment_area']

In [79]:
atx_school_zcta.head()

,YEAR,REGION_1,COUNTY NAME,DISTRICT,DISTRICT NAME,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT,...,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,zcta_area,geometry,overlap_area,pct_zcta_to_school,pct_school_from_zcta
0,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119,...,A,676653171537,19006305260,+31.4347032,-099.2818238,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
1,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81,...,A,676653171537,19006305260,+31.4347032,-099.2818238,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
2,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114,...,A,676653171537,19006305260,+31.4347032,-099.2818238,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
3,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112,...,A,676653171537,19006305260,+31.4347032,-099.2818238,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
4,2019-2020,13,TRAVIS COUNTY,227825,AUSTIN ACHIEVE PUBLIC SCHOOLS,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130,...,A,676653171537,19006305260,+31.4347032,-099.2818238,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08


In [80]:
atx_school_zcta.columns

Index(['YEAR', 'REGION_1', 'COUNTY NAME', 'DISTRICT', 'DISTRICT NAME',
       'CAMPUS', 'CAMPUS NAME', 'GRADE GROUP NAME', 'GRADE', 'ENROLLMENT',
       'SchoolNumber', 'catchment_area', 'ZCTA5CE10', 'GEOID10', 'CLASSFP10',
       'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10',
       'INTPTLON10', 'index_righ', 'REGION_2', 'DIVISION', 'STATEFP',
       'STATENS', 'GEOID', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT',
       'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'zcta_area', 'geometry',
       'overlap_area', 'pct_zcta_to_school', 'pct_school_from_zcta'],
      dtype='object')

In [81]:
atx_school_zcta_keep = atx_school_zcta[
    ['YEAR', 'COUNTY NAME', 'DISTRICT', 'CAMPUS', 'CAMPUS NAME', 
     'GRADE GROUP NAME', 'GRADE', 'ENROLLMENT', 
     'catchment_area', 'ZCTA5CE10', 'zcta_area', 'geometry', 'overlap_area', 
     'pct_zcta_to_school', 'pct_school_from_zcta']
]

In [82]:
atx_school_zcta_keep.head()

,YEAR,COUNTY NAME,DISTRICT,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT,catchment_area,ZCTA5CE10,zcta_area,geometry,overlap_area,pct_zcta_to_school,pct_school_from_zcta
0,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
1,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
2,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
3,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08
4,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08


In [83]:
atx_school_zcta_keep.dtypes

YEAR                      object
COUNTY NAME               object
DISTRICT                  object
CAMPUS                    object
CAMPUS NAME               object
GRADE GROUP NAME          object
GRADE                     object
ENROLLMENT                object
catchment_area           float64
ZCTA5CE10                 object
zcta_area                float64
geometry                geometry
overlap_area             float64
pct_zcta_to_school       float64
pct_school_from_zcta     float64
dtype: object

In [84]:
atx_school_zcta_keep['ENROLLMENT'].unique()

array(['119', '81', '114', '112', '130', '140', '166', '160', '104', '90',
       '60', '61', '69', '62', '55', '67', '28', '31', '18', '<10', '196',
       '191', '156', '136', '87', '86', '111', '118', '116', '56', '78',
       '77', '71', '72', '85', '75', '66', '70', '80', '59', '65', '79',
       '73', '98', '<90', '92', '110', '105', '29', '561', '<40', '48',
       '553', '533', '509', '462', '172', '117', '93', '64', '<60', '63',
       '83', '91', '95', '100', '102', '108', '<100', '99', '103', '109',
       '115', '127', '214', '154', '139', '128', '151', '184', '164',
       '141', '53', '58', '52', '36', '27', '20', '<50', '96', '146',
       '179', '201', '76', '<70', '82', '68', '107', '16', '271', '632',
       '547', '528', '482', '<180', '<170', '143', '57', '121', '<110',
       '125', '313', '301', '292', '978', '816', '714', '662', '14', '97',
       '106', '<20', '277', '330', '315', '939', '932', '755', '684',
       '13', '24', '94', '39', '22', '51', '89', '50',

In [85]:
atx_school_zcta_keep['ENROLLMENT'] = [float(i.replace('<', '')) for i in atx_school_zcta_keep['ENROLLMENT']]

/Users/kpierce/episimlab/esl-venv/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [86]:
atx_school_zcta_keep['count_school_from_zcta'] = atx_school_zcta_keep['ENROLLMENT'] * atx_school_zcta_keep['pct_school_from_zcta']


In [87]:
atx_school_zcta_keep.head()

,YEAR,COUNTY NAME,DISTRICT,CAMPUS,CAMPUS NAME,GRADE GROUP NAME,GRADE,ENROLLMENT,catchment_area,ZCTA5CE10,zcta_area,geometry,overlap_area,pct_zcta_to_school,pct_school_from_zcta,count_school_from_zcta
0,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Pre-kindergarten,119.0,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08,0.000002
1,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Kindergarten,81.0,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08,0.000001
2,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 1,114.0,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08,0.000002
3,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 2,112.0,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08,0.000002
4,2019-2020,TRAVIS COUNTY,227825,227825003,AUSTIN ACHIEVE EL,ELEMENTARY,Grade 3,130.0,3.129696e+08,78723,2.420059e+07,"POLYGON ((-10873579.294 3545223.829, -10873579...",4.205438,1.737742e-07,1.343721e-08,0.000002


In [88]:
max(atx_school_zcta_keep['count_school_from_zcta'])

757.5468220768211

In [89]:
atx_school_zcta_keep_final = atx_school_zcta_keep[atx_school_zcta_keep['count_school_from_zcta'] > 1]

In [90]:
print(atx_school_zcta_keep.shape)
print(atx_school_zcta_keep_final.shape)

(29418, 16)
(17692, 16)


Reformat the column names

In [91]:
atx_school_short = atx_school_zcta_keep_final[['ZCTA5CE10', 'CAMPUS NAME', 'GRADE', 'count_school_from_zcta']]
atx_school_short = atx_school_short.rename(
    columns={
        'ZCTA5CE10': 'i',
        'CAMPUS NAME': 'j',
        'count_school_from_zcta': 'n_ij'
    }
)

In [92]:
atx_school_short.head()

,i,j,GRADE,n_ij
198,78724,AUSTIN ACHIEVE EL,Pre-kindergarten,19.091282
199,78724,AUSTIN ACHIEVE EL,Kindergarten,12.994907
200,78724,AUSTIN ACHIEVE EL,Grade 1,18.289128
201,78724,AUSTIN ACHIEVE EL,Grade 2,17.968266
202,78724,AUSTIN ACHIEVE EL,Grade 3,20.856023


In [93]:
atx_school_short.GRADE.unique()

array(['Pre-kindergarten', 'Kindergarten', 'Grade 1', 'Grade 2',
       'Grade 3', 'Grade 4', 'Grade 9', 'Grade 10', 'Grade 11',
       'Grade 12', 'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8',
       'Early Education'], dtype=object)

In [94]:
ages_by_grade = {
    'Pre-kindergarten': '0-4', 
    'Kindergarten': '5-9', 
    'Grade 1': '5-9', 
    'Grade 2': '5-9',
    'Grade 3': '5-9', 
    'Grade 4': '5-9', 
    'Grade 9': '10-14', 
    'Grade 10': '15-17', 
    'Grade 11': '15-17',
    'Grade 12': '15-17', 
    'Grade 5': '10-14', 
    'Grade 6': '10-14', 
    'Grade 7': '10-14', 
    'Grade 8': '10-14',
    'Early Education': '0-4'
}


In [95]:
atx_school_short['age_group'] = [ages_by_grade[i] for i in atx_school_short['GRADE']]

In [96]:
atx_school_final = atx_school_short.drop('GRADE', axis=1)

In [97]:
atx_school_final = atx_school_final.groupby(['i', 'j', 'age_group']).sum('n_ij').reset_index()

In [98]:
atx_school_final.head()

,i,j,age_group,n_ij
0,75001,KIPP DESTINY MIDDLE,10-14,3.282118
1,75001,KIPP PLEASANT GROVE MIDDLE,10-14,6.871934
2,75001,KIPP TRUTH ACADEMY,10-14,9.165687
3,75115,KIPP DESTINY MIDDLE,10-14,1.106815
4,75115,KIPP OAK CLIFF ACADEMY,10-14,4.168768


# Safegraph social distancing metrics combined with census for travel ages > 13

- n$_{ij}$ = n trips from ZCTA$_{i}$ to ZCTA$_{j}$ (from SafeGraph social distancing metrics)
- n$_{i}$ = total devices associated with ZCTA$_{i}$ (from SafeGraph social dist)
- pr(travel$_{ij}$) = n$_{ij}$/n$_{i}$  
- N$_{ij}$ travel by age = pr(travel$_{ij}$) * N$_{age}$

In [99]:
travel20 = pd.read_csv('/Users/kpierce/COVID19/safegraph_mobility/cbg20_travel_edges.csv',
                      dtype={'location_1': str, 'location_2': str})

In [100]:
with open('/Users/kpierce/COVID19/safegraph_mobility/src/transform/data/crosswalks/cbg_to_zcta.json', 'r') as f:
    cbg_zcta = json.load(f)

In [101]:
travel20['zcta1'] = [cbg_zcta[i] for i in travel20['location_1']]
travel20['zcta2'] = [cbg_zcta[i] for i in travel20['location_2']]

In [102]:
travel20.head()

,Unnamed: 0,Unnamed: 0.1,location_1,location_2,count_loc1_to_loc2,count_loc2_to_loc1,date,day_of_week,day_type,week_of_year,month_of_year,geography,zcta1,zcta2
0,0,0,480219501001,480219501001,160.0,160.0,2020-03-30,Monday,weekday,14,3,block_group,78621,78621
1,1,1,480219501001,480219501002,0.0,1.0,2020-03-30,Monday,weekday,14,3,block_group,78621,78621
2,2,2,480219501001,480219501004,0.0,2.0,2020-03-30,Monday,weekday,14,3,block_group,78621,78621
3,3,3,480219501001,480219501005,2.0,1.0,2020-03-30,Monday,weekday,14,3,block_group,78621,78650
4,4,4,480219501001,480219502001,6.0,1.0,2020-03-30,Monday,weekday,14,3,block_group,78621,78621


Calculate n$_{ij}$:

In [103]:
travel20_agg = travel20.groupby(['zcta1', 'zcta2', 'date']).agg({'count_loc1_to_loc2': sum, 'count_loc2_to_loc1': sum}).reset_index()


In [104]:
travel20_agg[(travel20_agg['zcta1']=='78621') & (travel20_agg['zcta2']=='78621') & (travel20_agg['date']=='2020-03-30')]


,zcta1,zcta2,date,count_loc1_to_loc2,count_loc2_to_loc1
299505,78621,78621,2020-03-30,1599.0,1599.0


Calculate n$_{i}$ (also checking that the totals are the same from both perspectives -- zcta as source and zcta as destination):

In [105]:
travel20_by_source1 = travel20.groupby(['zcta1', 'date'])['count_loc1_to_loc2'].sum().reset_index()
travel20_by_source2 = travel20.groupby(['zcta2', 'date'])['count_loc2_to_loc1'].sum().reset_index()

In [106]:
travel20_by_source1 = travel20_by_source1.rename(columns={'zcta1': 'i', 'count_loc1_to_loc2': 'n_i'})
travel20_by_source2 = travel20_by_source2.rename(columns={'zcta2': 'i', 'count_loc2_to_loc1': 'n_i'})

In [107]:
travel20_by_source1.head()

,i,date,n_i
0,76511,2020-01-01,96.0
1,76511,2020-01-02,124.0
2,76511,2020-01-03,163.0
3,76511,2020-01-04,112.0
4,76511,2020-01-05,97.0


In [108]:
travel20_by_source2.head()

,i,date,n_i
0,76511,2020-01-01,96.0
1,76511,2020-01-02,124.0
2,76511,2020-01-03,163.0
3,76511,2020-01-04,112.0
4,76511,2020-01-05,97.0


In [109]:
travel20_by_source1.equals(travel20_by_source2)

True

Calculate pr(travel$_{ij})$:

In [110]:
travel20_zcta1_2 = travel20_agg[['zcta1', 'zcta2', 'date', 'count_loc1_to_loc2']]
travel20_zcta1_2 = travel20_zcta1_2.rename(columns={'zcta1': 'i', 'zcta2': 'j', 'count_loc1_to_loc2': 'n_ij'})

travel20_zcta2_1 = travel20_agg[['zcta2', 'zcta1', 'date', 'count_loc2_to_loc1']]
travel20_zcta2_1 = travel20_zcta2_1.rename(columns={'zcta2': 'i', 'zcta1': 'j', 'count_loc2_to_loc1': 'n_ij'})


In [111]:
travel20_src_dest = pd.concat([travel20_zcta1_2, travel20_zcta2_1])

In [112]:
travel20_src_dest.head()

,i,j,date,n_ij
0,76511,76511,2020-01-01,56.0
1,76511,76511,2020-01-02,57.0
2,76511,76511,2020-01-03,60.0
3,76511,76511,2020-01-04,54.0
4,76511,76511,2020-01-05,56.0


In [113]:
travel20_agg_pr = pd.merge(travel20_src_dest, travel20_by_source1, on=['i', 'date'], how='left')

In [114]:
travel20_agg_pr.head()

,i,j,date,n_ij,n_i
0,76511,76511,2020-01-01,56.0,96.0
1,76511,76511,2020-01-02,57.0,124.0
2,76511,76511,2020-01-03,60.0,163.0
3,76511,76511,2020-01-04,54.0,112.0
4,76511,76511,2020-01-05,56.0,97.0


In [115]:
travel20_agg_pr['pr(travel_ij)'] = travel20_agg_pr['n_ij'] / travel20_agg_pr['n_i']

In [116]:
travel20_agg_pr.head()

,i,j,date,n_ij,n_i,pr(travel_ij)
0,76511,76511,2020-01-01,56.0,96.0,0.583333
1,76511,76511,2020-01-02,57.0,124.0,0.459677
2,76511,76511,2020-01-03,60.0,163.0,0.368098
3,76511,76511,2020-01-04,54.0,112.0,0.482143
4,76511,76511,2020-01-05,56.0,97.0,0.577320


In [117]:
max(travel20_agg_pr['pr(travel_ij)'])

0.8952933693750994

Combine with age-specific population data.

In [126]:
travel20_by_age = pd.merge(travel20_agg_pr, census, left_on='i', right_on='GEOID', how='left')

In [127]:
travel20_by_age['N_ij'] = travel20_by_age['pr(travel_ij)'] * travel20_by_age['estimate']

## Add risk to mock up census data

In [129]:
census['high'] = census['estimate'] * 0.1
census['low'] = census['estimate'] - census['high']

In [130]:
mock_census = census.melt(id_vars=['GEOID', 'age_group'], value_vars=['high', 'low'], var_name='risk', value_name='n')

In [131]:
mock_census.head()

,GEOID,age_group,risk,n
0,75001,0-4,high,79.4
1,75001,10-14,high,40.1
2,75001,15-17,high,27.7
3,75001,18-49,high,942.0
4,75001,5-9,high,72.6


In [132]:
mock_census.shape

(27090, 4)

In [138]:
import xarray as xr
from copy import deepcopy

In [134]:
mock_census = mock_census.rename(columns={'GEOID': 'vertex', 'age_group':'age'})

In [135]:
mock_census['compt'] = 'S'

In [136]:
other_compt = ['S', 'V', 'E', 'Ev', 'Pa', 'Py', 'Ia', 'Iy', 'Ih', 'R', 'D']

In [139]:
mock_compt_list = []
for c in other_compt:
    mock_copy = deepcopy(mock_census)
    mock_copy['compt'] = c
    mock_copy['n'] = 0
    mock_compt_list.append(mock_copy)

mock_census_full = pd.concat(mock_compt_list)

In [140]:
mock_census_full['compt'].unique()

array(['S', 'V', 'E', 'Ev', 'Pa', 'Py', 'Ia', 'Iy', 'Ih', 'R', 'D'],
      dtype=object)

In [141]:
mock_census_full.tail()

,vertex,age,risk,n,compt
27085,79942,15-17,low,0,D
27086,79942,18-49,low,0,D
27087,79942,5-9,low,0,D
27088,79942,50-64,low,0,D
27089,79942,65+,low,0,D


In [142]:
mock_census_full.to_csv('/Users/kpierce/COVID19/SchoolCatchmentDemo/mock_initial_state_census_20211203.csv')

In [143]:
mock_census_xr = mock_census_full.set_index(['vertex', 'age', 'risk', 'compt']).to_xarray().to_array()

In [144]:
mock_census_xr

<xarray.DataArray (variable: 1, vertex: 1935, age: 7, risk: 2, compt: 11)>
array([[[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         ...,

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]],

...

        [[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         ...,

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],

         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]]])
Coordinates:
  * vertex    (vertex) object '75001' '75002' '75006' ... '79938' '79942'
  * age       (age) object '0-4' '10-14' '15-17' '18-49' '5-9' '50-64' '65+'
  * risk      (risk) object 'high' 'low'
  * compt     (compt) object 'D' 'E' 'Ev' 'Ia' 'Ih' 'Iy' 'Pa' 'Py' 'R' 'S' 'V'
  * variable  (variable) <U1 'n'

In [145]:
mock_census_xr.loc[dict(compt='Ia', age='18-49', risk='low', vertex='78712')]

<xarray.DataArray (variable: 1)>
array([0])
Coordinates:
    vertex    <U5 '78712'
    age       <U5 '18-49'
    risk      <U3 'low'
    compt     <U2 'Ia'
  * variable  (variable) <U1 'n'

In [146]:
from numpy import random

In [147]:
random.choice(mock_census_xr['age'])

'65+'

In [148]:
random.choice(mock_census_xr['risk'])

'low'

In [149]:
random.choice(mock_census_xr['vertex'])

'77963'

# Combine ZCTA to school estimates with Safegraph travel

For simplicity of demo, ages 5-9 = elementary school, 10-14 = middle school, and 15-17 = high school. This is *NOT* true for all schools.

Further, assume all kids go to school, and that safegraph only includes ages 18 and up. Also *NOT* actually true.

In [150]:
travel20.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'location_1', 'location_2',
       'count_loc1_to_loc2', 'count_loc2_to_loc1', 'date', 'day_of_week',
       'day_type', 'week_of_year', 'month_of_year', 'geography', 'zcta1',
       'zcta2'],
      dtype='object')

In [151]:
travel20_dates = travel20[['date', 'day_type']].drop_duplicates()

In [152]:
travel20_dates.head()

,date,day_type
0,2020-03-30,weekday
83937,2020-09-27,weekend
178587,2020-03-07,weekend
317204,2020-09-10,weekday
435267,2020-07-01,weekday


In [153]:
travel20_by_age_day_type = pd.merge(travel20_by_age, travel20_dates, on='date', how='left')

In [154]:
travel20_safegraph = travel20_by_age_day_type[
    (travel20_by_age_day_type['age_group'].isin(set(['18-49', '50-64', '65+']))) | \
    (travel20_by_age_day_type['day_type'] == 'weekend')
]


In [155]:
travel20_safegraph[travel20_safegraph['day_type'] == 'weekend']

,i,j,date,n_ij,n_i,pr(travel_ij),GEOID,age_group,estimate,N_ij,day_type
21,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,0-4,93,44.839286,weekend
22,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,10-14,106,51.107143,weekend
23,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,15-17,103,49.660714,weekend
24,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,18-49,1253,604.125000,weekend
25,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,5-9,189,91.125000,weekend
...,...,...,...,...,...,...,...,...,...,...,...
28012637,78957,78957,2020-12-27,781.0,1122.0,0.696078,78957,15-17,353,245.715686,weekend
28012638,78957,78957,2020-12-27,781.0,1122.0,0.696078,78957,18-49,3290,2290.098039,weekend
28012639,78957,78957,2020-12-27,781.0,1122.0,0.696078,78957,5-9,607,422.519608,weekend
28012640,78957,78957,2020-12-27,781.0,1122.0,0.696078,78957,50-64,3938,2741.156863,weekend


We also need to check that the census estimates and the school enrollment estimates are concordant (we don't lose children on the weekends or on the weekdays)

In [156]:
atx_school_zip_sums = atx_school_final.groupby(['i', 'age_group']).sum('n_ij').reset_index()

In [157]:
atx_school_zip_sums.head()

,i,age_group,n_ij
0,75001,10-14,19.319740
1,75115,10-14,18.374344
2,75115,15-17,5.230260
3,75125,10-14,161.510988
4,75125,15-17,31.839786


In [158]:
travel20_safegraph_kids = travel20_safegraph[(travel20_safegraph['age_group'].isin(set(['0-4', '5-9', '10-14', '15-17']))) & (travel20_safegraph['i'] == travel20_safegraph['j'])]


In [159]:
travel20_safegraph_kids.head()

,i,j,date,n_ij,n_i,pr(travel_ij),GEOID,age_group,estimate,N_ij,day_type
21,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,0-4,93,44.839286,weekend
22,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,10-14,106,51.107143,weekend
23,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,15-17,103,49.660714,weekend
25,76511,76511,2020-01-04,54.0,112.0,0.482143,76511,5-9,189,91.125000,weekend
28,76511,76511,2020-01-05,56.0,97.0,0.577320,76511,0-4,93,53.690722,weekend


In [160]:
travel20_school = pd.merge(travel20_safegraph_kids, atx_school_zip_sums, on=['i', 'age_group'], how='left')

In [161]:
travel20_school[~travel20_school['n_ij_y'].isna()]

,i,j,date,n_ij_x,n_i,pr(travel_ij),GEOID,age_group,estimate,N_ij,day_type,n_ij_y
380,76527,76527,2020-01-04,297.0,618.0,0.480583,76527,0-4,198,95.155340,weekend,71.204916
381,76527,76527,2020-01-04,297.0,618.0,0.480583,76527,10-14,192,92.271845,weekend,616.634569
382,76527,76527,2020-01-04,297.0,618.0,0.480583,76527,15-17,166,79.776699,weekend,357.448676
383,76527,76527,2020-01-04,297.0,618.0,0.480583,76527,5-9,262,125.912621,weekend,519.795884
384,76527,76527,2020-01-05,282.0,492.0,0.573171,76527,0-4,198,113.487805,weekend,71.204916
...,...,...,...,...,...,...,...,...,...,...,...,...
65730,78953,78953,2020-12-20,100.0,218.0,0.458716,78953,15-17,0,0.000000,weekend,6.785638
65733,78953,78953,2020-12-26,104.0,205.0,0.507317,78953,10-14,0,0.000000,weekend,8.334534
65734,78953,78953,2020-12-26,104.0,205.0,0.507317,78953,15-17,0,0.000000,weekend,6.785638
65737,78953,78953,2020-12-27,99.0,196.0,0.505102,78953,10-14,0,0.000000,weekend,8.334534


Under this set of assumptions, a handful of kids disappear over the weekend: 78953 (and surely other zip codes as well) are estimated to contribute 6-8 children to schools based on areal weighted enrollment interpolation, but are not estimated to have any population in those age groups.

I am going to move forward with the demo but a proper publication quality analysis would resolve this issue.

# Final tidying and concatenation

In [386]:
travel20_safegraph_final = travel20_safegraph[['i', 'j', 'date', 'age_group', 'N_ij']]
travel20_safegraph_final['destination_type'] = 'local'
travel20_safegraph_final = travel20_safegraph_final[(travel20_safegraph_final['date'] >= '2020-01-07') & \
                                                    (travel20_safegraph_final['date'] <= '2020-05-28')]
travel20_safegraph_final.head()

/var/folders/hn/w8yttt8x7k5141wgqdcfv7fh0000gr/T/ipykernel_86503/4010361493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel20_safegraph_final['destination_type'] = 'local'


,i,j,date,age_group,N_ij,destination_type
45,76511,76511,2020-01-07,18-49,522.083333,local
47,76511,76511,2020-01-07,50-64,291.666667,local
48,76511,76511,2020-01-07,65+,25.416667,local
52,76511,76511,2020-01-08,18-49,526.068702,local
54,76511,76511,2020-01-08,50-64,293.893130,local


In [387]:
travel20_safegraph.age_group.unique()

array(['18-49', '50-64', '65+', '0-4', '10-14', '15-17', '5-9'],
      dtype=object)

In [388]:
travel20_safegraph_final.age_group.unique()

array(['18-49', '50-64', '65+', '0-4', '10-14', '15-17', '5-9'],
      dtype=object)

In [389]:
weekdays = travel20_dates[(travel20_dates['day_type'] == 'weekday') & \
                          (travel20_dates['date'] >= '2020-01-07') & \
                          (travel20_dates['date'] <= '2020-05-28')]

In [390]:
weekdays

,date,day_type
0,2020-03-30,weekday
661556,2020-03-31,weekday
876916,2020-03-06,weekday
1143632,2020-03-05,weekday
1771147,2020-03-04,weekday
...,...,...
35644009,2020-03-17,weekday
35857111,2020-03-20,weekday
36034118,2020-03-23,weekday
36360512,2020-04-09,weekday


In [391]:
from copy import deepcopy

In [392]:
repeat_weekdays = []
for w in weekdays['date'].unique():
    schoolday = deepcopy(atx_school_final)
    schoolday['date'] = w
    repeat_weekdays.append(schoolday)
atx_school_daily_final = pd.concat(repeat_weekdays)

In [393]:
atx_school_daily_final['destination_type'] = 'contextual'
atx_school_daily_final = atx_school_daily_final.rename(columns={'n_ij': 'N_ij'})
atx_school_daily_final.head()

,i,j,age_group,N_ij,date,destination_type
0,75001,KIPP DESTINY MIDDLE,10-14,3.282118,2020-03-30,contextual
1,75001,KIPP PLEASANT GROVE MIDDLE,10-14,6.871934,2020-03-30,contextual
2,75001,KIPP TRUTH ACADEMY,10-14,9.165687,2020-03-30,contextual
3,75115,KIPP DESTINY MIDDLE,10-14,1.106815,2020-03-30,contextual
4,75115,KIPP OAK CLIFF ACADEMY,10-14,4.168768,2020-03-30,contextual


In [542]:
school_demo_final_travel = pd.concat([atx_school_daily_final, travel20_safegraph_final])

In [543]:
school_demo_final_travel.head()

,i,j,age_group,N_ij,date,destination_type
0,75001,KIPP DESTINY MIDDLE,10-14,3.282118,2020-03-30,contextual
1,75001,KIPP PLEASANT GROVE MIDDLE,10-14,6.871934,2020-03-30,contextual
2,75001,KIPP TRUTH ACADEMY,10-14,9.165687,2020-03-30,contextual
3,75115,KIPP DESTINY MIDDLE,10-14,1.106815,2020-03-30,contextual
4,75115,KIPP OAK CLIFF ACADEMY,10-14,4.168768,2020-03-30,contextual


In [544]:
school_demo_final_travel.shape

(7351577, 6)

In [545]:
sdft_dedup = school_demo_final_travel.drop_duplicates()

In [546]:
sdft_dedup.shape

(3847644, 6)

In [547]:
check_ages_dates = sdft_dedup[['date', 'age_group']].drop_duplicates().groupby(['date'])['age_group'].count()


In [548]:
check_ages_dates.head(20)

date
2020-01-07    7
2020-01-08    7
2020-01-09    7
2020-01-10    7
2020-01-11    7
2020-01-12    7
2020-01-13    7
2020-01-14    7
2020-01-15    7
2020-01-16    7
2020-01-17    7
2020-01-18    7
2020-01-19    7
2020-01-20    7
2020-01-21    7
2020-01-22    7
2020-01-23    7
2020-01-24    7
2020-01-25    7
2020-01-26    7
Name: age_group, dtype: int64

In [414]:
min(check_ages_dates)

7

In [415]:
max(check_ages_dates)

7

In [467]:
school_demo_final_travel = school_demo_final_travel.rename(
    columns={'i': 'source', 'j': 'destination', 'age_group': 'age', 'N_ij': 'n'})
school_demo_final_travel.to_csv('/Users/kpierce/COVID19/SchoolCatchmentDemo/spring_semester_2020_travel.csv')

In [527]:
school_demo_final_travel[school_demo_final_travel['date']=='2020-01-07']['age'].unique()

array(['10-14', '15-17', '5-9', '0-4', '18-49', '50-64', '65+'],
      dtype=object)

# Mock polymod contacts

In [419]:
polymod = pd.read_csv('/Users/kpierce/episimlab/data/polymod_contacts.csv')

In [455]:
child_contacts1 = polymod[(polymod['age1']=='5-17') & (polymod['age2']!='5-17')]
mock_age1 = pd.DataFrame.from_dict({
    'age1': ['5-9' for i in range(len(child_contacts1['age1']))] + \
            ['10-14' for i in range(len(child_contacts1['age1']))] + \
            ['15-17' for i in range(len(child_contacts1['age1']))],
    'age2': list(child_contacts1['age2']) + list(child_contacts1['age2']) + list(child_contacts1['age2']),
    'daily_per_capita_contacts': list(child_contacts1['daily_per_capita_contacts']) + \
                                 list(child_contacts1['daily_per_capita_contacts']) + \
                                 list(child_contacts1['daily_per_capita_contacts'])
})

In [456]:
child_contacts1

,Unnamed: 0,age1,age2,daily_per_capita_contacts
1,1,5-17,<5,0.597341
11,11,5-17,18-49,5.406032
16,16,5-17,50-64,0.736618
21,21,5-17,65+,0.226147


In [457]:
mock_age1

,age1,age2,daily_per_capita_contacts
0,5-9,<5,0.597341
1,5-9,18-49,5.406032
2,5-9,50-64,0.736618
3,5-9,65+,0.226147
4,10-14,<5,0.597341
5,10-14,18-49,5.406032
6,10-14,50-64,0.736618
7,10-14,65+,0.226147
8,15-17,<5,0.597341
9,15-17,18-49,5.406032


In [452]:
child_contacts2 = polymod[(polymod['age2']=='5-17') & (polymod['age1']!='5-17')]
mock_age2 = pd.DataFrame.from_dict({
    'age1': list(child_contacts2['age1']) + list(child_contacts2['age1']) + list(child_contacts2['age1']),
    'age2': ['5-9' for i in range(len(child_contacts2['age1']))] + \
            ['10-14' for i in range(len(child_contacts2['age1']))] + \
            ['15-17' for i in range(len(child_contacts2['age1']))],
    
    'daily_per_capita_contacts': list(child_contacts2['daily_per_capita_contacts']) + \
                                 list(child_contacts2['daily_per_capita_contacts']) + \
                                 list(child_contacts2['daily_per_capita_contacts'])
})

In [453]:
child_contacts2

,Unnamed: 0,age1,age2,daily_per_capita_contacts
5,5,<5,5-17,2.164117
7,7,18-49,5-17,2.431392
8,8,50-64,5-17,1.885100
9,9,65+,5-17,0.892909


In [454]:
mock_age2

,age1,age2,daily_per_capita_contacts
0,<5,5-9,2.164117
1,18-49,5-9,2.431392
2,50-64,5-9,1.885100
3,65+,5-9,0.892909
4,<5,10-14,2.164117
5,18-49,10-14,2.431392
6,50-64,10-14,1.885100
7,65+,10-14,0.892909
8,<5,15-17,2.164117
9,18-49,15-17,2.431392


In [458]:
polymod_keep = polymod[(polymod['age1']!='5-17') & (polymod['age2']!='5-17')]

In [459]:
polymod_keep

,Unnamed: 0,age1,age2,daily_per_capita_contacts
0,0,<5,<5,2.160941
2,2,18-49,<5,0.382203
3,3,50-64,<5,0.352397
4,4,65+,<5,0.189756
10,10,<5,18-49,4.119176
12,12,18-49,18-49,10.236284
13,13,50-64,18-49,6.706929
14,14,65+,18-49,2.390370
15,15,<5,50-64,0.808566
17,17,18-49,50-64,1.700688


In [462]:
mock_polymod_final = pd.concat([polymod_keep, mock_age1, mock_age2]).drop('Unnamed: 0', axis=1)

In [529]:
mock_polymod_final['age1'] = [i if i != '<5' else '0-4' for i in mock_polymod_final['age1']]
mock_polymod_final['age2'] = [i if i != '<5' else '0-4' for i in mock_polymod_final['age2']]

In [530]:
mock_polymod_final

,age1,age2,daily_per_capita_contacts
0,0-4,0-4,2.160941
2,18-49,0-4,0.382203
3,50-64,0-4,0.352397
4,65+,0-4,0.189756
10,0-4,18-49,4.119176
12,18-49,18-49,10.236284
13,50-64,18-49,6.706929
14,65+,18-49,2.390370
15,0-4,50-64,0.808566
17,18-49,50-64,1.700688


In [531]:
mock_polymod_final.to_csv('/Users/kpierce/COVID19/SchoolCatchmentDemo/mock_polymod_contacts.csv')

In [537]:
sorted(mock_polymod_final['age1'].unique())

['0-4', '10-14', '15-17', '18-49', '5-9', '50-64', '65+']